In [19]:
import socket
import os
import tqdm

#port meaning sent packet to portXXX, i.e. send to server ip and with it service port
def client(host='127.0.0.1', port=12346, file_path='received_image.jpg'):
    buffer_size = 1024
    ack_message = b'ACK'
    timeout = 2  # seconds

    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as client_socket:
        client_socket.settimeout(timeout)
        file_size = os.path.getsize(file_path)
        progress = tqdm.tqdm(range(file_size), f"Sending {file_path}", unit="B", unit_scale=True, unit_divisor=1024)

        with open(file_path, 'rb') as f:
            seq_num = 0
            while True:
                data = f.read(buffer_size)
                if not data:
                    break

                while True:
                    packet = seq_num.to_bytes(4, 'big') + data
                    client_socket.sendto(packet, (host, port))

                    try:
                        response, _ = client_socket.recvfrom(buffer_size + 4)
                        if response.startswith(ack_message) and int.from_bytes(response[3:], 'big') == seq_num:
                            progress.update(len(data))
                            seq_num += 1
                            break
                    except socket.timeout:
                        print(f"Timeout, resending packet {seq_num}")
            
        print("Image sent.")

if __name__ == "__main__":
    client('127.0.0.1', 12346, 'Screenshot_20240527_142500.jpg')









Sending Screenshot_20240519_120249.jpg:   0%|                                               | 0.00/693k [00:00<?, ?B/s]






Sending Screenshot_20240519_120249.jpg:  14%|█████▍                                 | 96.0k/693k [00:00<00:00, 955kB/s]






Sending Screenshot_20240519_120249.jpg:  30%|███████████▌                           | 206k/693k [00:00<00:00, 1.05MB/s]






Sending Screenshot_20240519_120249.jpg:  46%|█████████████████▉                     | 319k/693k [00:00<00:00, 1.09MB/s]






Sending Screenshot_20240519_120249.jpg:  65%|█████████████████████████▎             | 451k/693k [00:00<00:00, 1.17MB/s]






Sending Screenshot_20240519_120249.jpg: 100%|███████████████████████████████████████| 693k/693k [00:00<00:00, 1.20MB/s]

Image sent.
